🔧 Step 1: Class Design (OOP Structure)

We'll start with 4 core classes, which satisfy OOP, inheritance, abstraction, and polymorphism requirements.

🔹 Route (Base Class)
Holds basic route details.

In [1]:
class Route:
    def __init__(self, route_id, distance, duration, cost, expression, variables):
        self.route_id = route_id
        self.distance = distance  # in km
        self.duration = duration  # in minutes
        self.cost = cost          # in currency
        self.expression = expression  # logical expression (e.g. "p and not q")
        self.variables = variables    # dictionary of {'p': True, 'q': False}

    def evaluate_expression(self):
        try:
            return eval(self.expression, {}, self.variables)
        except Exception as e:
            print(f"Error evaluating expression for Route {self.route_id}: {e}")
            return False

🔹 ScenicRoute (Subclass of Route)
Demonstrates inheritance and adds a new property.

In [3]:
class ScenicRoute(Route):
    def __init__(self, route_id, distance, duration, cost, expression, variables, has_scenic_view):
        super().__init__(route_id, distance, duration, cost, expression, variables)
        self.has_scenic_view = has_scenic_view